# Lecture 2: Discrete Markov process

In [ ]:

%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx

import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl

In [ ]:
def get_filename(filename: str, lecture_id: int = 1, file_extension: str = '.png') -> str:
    return f"L{lecture_id}_{filename}{file_extension}"

outdir = '../figures/'
lecture_id = 2

In [ ]:
'''
------------------------------------------
            SETTINGS
------------------------------------------
'''
plt.style.use('fivethirtyeight')
plt.style.use('seaborn-v0_8-white')
plt.rcParams['font.family'] = 'PT Sans'
# plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 14
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['figure.titlesize'] = 12

dpi = 100

# 1. Example discrete Markov process chain

Let's consider an example of a process modeling a server. Its status can be one of **three** possible states:
- Operational
- Overloaded 
- Critical

The transition probability matrix $P$ is given as in the code below (you can play with it).

In [ ]:
states = ["Operational", "Overloaded", "Critical"]
P = np.array([
    [0.55, 0.3, 0.15],  # From Operational to [Operational, Overloaded, Critical]
    [0.4, 0.4, 0.2],  
    [0.1, 0.5, 0.4]   
])

We can visualize it as a matrix visualization

In [ ]:
fs = 20
plt.figure(figsize=(6,3))
plt.imshow(P, vmax=1,vmin=0, cmap='Blues')

for (j,i),label in np.ndenumerate(P):
    plt.text(i,j,f"{label:.1f}",ha='center',va='center', c='black', fontsize = fs)
    plt.text(i,j,f"{label:.1f}",ha='center',va='center', c='black', fontsize = fs)
plt.axis('off')
plt.colorbar()

## 1.1  Check that it is a valid probability
This is always the first step to take, as a sanity check.

We ask that the row-sum of $P$ equals 1:
- $\sum_j P_{ij} =1$, $\forall i$

In [ ]:
assert np.all(np.isclose(np.sum(P,1),1))

## 1.2 Visualize into a Markov graph
Now that we have $P$, we can visualize using a **graph** (or network).  
This will be handy to use all the methods and tools available from network modeling.   
In particular, we will use the [`networkx`](https://networkx.org/) python module.

In [ ]:
filename = 'markov_chain_ex1'
filename = get_filename(filename,lecture_id=lecture_id)

outfile = filename
outfile

Let's first define a function to takes in input the matrix $P$ and builds a `networkx` graph object `G` with: 
- **nodes**: states of the Markov chain  
- **edge** weights: entries of $P$

In [ ]:
def get_graph_from_transition(P: np.ndarray,states: list) -> nx.MultiDiGraph():
    G = nx.MultiDiGraph()
    assert P.shape[0] == len(states) 
    for start_idx, node_start in enumerate(states):
        for end_idx, node_end in enumerate(states):
            value = P[start_idx][end_idx]
            if value != 0:
                G.add_edge(node_start,node_end, weight=value)
    return G

Let's visualize this graph.

In [ ]:
G = get_graph_from_transition(P,states)


pos = nx.spring_layout(G, seed=10)
fig, ax = plt.subplots()
nx.draw_networkx_nodes(G, pos, node_size=1000, edgecolors='black', node_color='white')
nx.draw_networkx_labels(G, pos, font_size=12)

arc_rad = 0.2

edges = nx.draw_networkx_edges(G, pos, ax=ax, connectionstyle=f'arc3, rad = {arc_rad}', edge_cmap=cm.Blues, width=5,
    edge_color=[G[nodes[0]][nodes[1]][0]['weight'] for nodes in G.edges])

pc = mpl.collections.PatchCollection(edges, cmap=cm.Blues)

ax = plt.gca()
ax.set_axis_off()
plt.colorbar(pc, ax=ax)
plt.tight_layout()
# plt.show()
if outfile is not None:
    
    plt.savefig(f"{outdir}{outfile}", dpi=dpi, format=None, metadata=None,
                bbox_inches='tight', pad_inches=0.1,
                facecolor='auto', edgecolor='auto',
                backend=None
                )
    print(f"Figure saved in {outdir}{outfile}")
        
plt.show()

Notice the existence of **self-loops** for the diagonal entries $P_{ii}$ and that **edge colors** denote the magnitude of $P_{ij}$

## 1.3 Irreducible vs reducible

#### Q1: is this irreducible?

We can use the network property of a graph being **strongly connected** to check this.

In [ ]:
print(f"Is irreducible?\n{nx.is_strongly_connected(G)}!")

#### Q2: how do you make it reducible?

You need to break the possibility of reaching any state from any other state...

In [ ]:
P_red = np.array([
    [0.55, 0.45, 0.0],  
    [0.48, 0.48, 0.04],  
    [0.0, 0.0, 1.]   # Once you are in Critical you cannot go back to any other states. You are stuck there
])
assert np.all(np.isclose(np.sum(P_red,1),1))

Let's check again if it is irreducible

In [ ]:
G_red = get_graph_from_transition(P_red,states)
print(f"Is irreducible?\n{nx.is_strongly_connected(G_red)}!")

# 2. Simulate a Markov Chain
We are now ready to play with the Markov chain and simulate one.  
First, it is convenient to build a function that the current state and simulates the next.

In [ ]:
# Simulate transitions
def next_state(current_state, transition_matrix):
    assert np.all(np.isclose(np.sum(transition_matrix,1),1))
    N = transition_matrix.shape[0]
    assert 0 <= current_state < N
    return np.random.choice(np.arange(N), p=transition_matrix[current_state])

Now we can use this to run it for various time steps.

In [ ]:
current_state = 0 # initial time step
days = 10 # number of timesteps to simulate
state_forecast = [states[current_state]]

fig, ax = plt.subplots(1,days,figsize=(6 * days, 5))

# Set initial state
current_state = 0  # Starting state
print(f"Starting state = {states[current_state]}")
for i in range(days):

    old_state = current_state
    node_color = ['white' for i in range(len(states))]
    node_color[old_state] = 'r'
    current_state = next_state(current_state, P)
    state_forecast.append(states[current_state])

    nx.draw_networkx_nodes(G, pos, node_size=3000, edgecolors='black', node_color=node_color,ax=ax[i],alpha=0.8)
    nx.draw_networkx_labels(G, pos, font_size=12, ax=ax[i])

    edge_color = ['black' for e in G.edges()]
    width = [1 for e in G.edges]
    for idx, (u,v) in enumerate(G.edges()):
        if (u == states[old_state]) & (v == states[current_state]):
            edge_color[idx] = 'r'
            width[idx] = 10
    edges = nx.draw_networkx_edges(G, pos, ax=ax[i], connectionstyle=f'arc3, rad = {arc_rad}', edge_cmap=cm.Blues, width=width,
                edge_color=edge_color, arrows=True, arrowsize=10)
    
    ax[i].set_axis_off()
    ax[i].set_title(f"t = {i}", fontsize=50)

plt.tight_layout()


In [ ]:
print(state_forecast[1:])

# 3. Number of Visits and Recurrent vs Transient
We can check what is the expected value of the number of visits to node $j$, given we start at $i$:

$\mathbb{E} [N_{i}(j)] = \frac{r_{ij}}{1-r_{jj}} \, = \sum_{n=1}^{\infty}(P^{n})_{ij}$

For this, we can compute the RHS using the power of a matrix

In [ ]:
from numpy.linalg import matrix_power

In [ ]:
P

First, let's take a look at $P^n$, tuning $n$ from small to large values

In [ ]:
max_N = 1000
P_to_the_N = matrix_power(P, max_N)
P_to_the_N

In [ ]:
fs = 20
plt.figure(figsize=(6,3))
fig, ax = plt.subplots(1,2,figsize=(6,3))

# Original transition matrix
ax[0].imshow(P, vmax=1,vmin=0, cmap='Blues')

for (j,i),label in np.ndenumerate(P):
    ax[0].text(i,j,f"{label:.2f}",ha='center',va='center', c='black', fontsize = fs)
    ax[0].text(i,j,f"{label:.2f}",ha='center',va='center', c='black', fontsize = fs)

ax[0].axis('off')
ax[0].set_title('P')
# n-step transition matrix
ax[1].imshow(P_to_the_N, vmax=1,vmin=0, cmap='Blues')

for (j,i),label in np.ndenumerate(P_to_the_N):
    ax[1].text(i,j,f"{label:.2f}",ha='center',va='center', c='black', fontsize = fs)
    ax[1].text(i,j,f"{label:.2f}",ha='center',va='center', c='black', fontsize = fs)
    
ax[1].axis('off')
ax[1].set_title(r'$P^n$' +f", n = {max_N}" )
# pc.colorbar()

#### Q3: play with `max_N`, what do you see?

Now we are ready to take the **sum** of this over various $n$

In [ ]:
ref_P = P.copy()
max_N = 10

sumP_to_the_N = sum([matrix_power(ref_P, n) for n in range(1,max_N+1)])
P_to_the_N = matrix_power(ref_P, max_N)

sumP_to_the_N

In [ ]:
fs = 20
# plt.figure(figsize=(12,4))
fig, ax = plt.subplots(1,3,figsize=(12,4))

# Original transition matrix
ax[0].imshow(ref_P, vmax=1,vmin=0, cmap='Blues')

for (j,i),label in np.ndenumerate(P):
    ax[0].text(i,j,f"{label:.2f}",ha='center',va='center', c='black', fontsize = fs)
    ax[0].text(i,j,f"{label:.2f}",ha='center',va='center', c='black', fontsize = fs)

ax[0].axis('off')
ax[0].set_title('P')

# n-step transition matrix
ax[1].imshow(P_to_the_N, cmap='Blues')

for (j,i),label in np.ndenumerate(P_to_the_N):
    ax[1].text(i,j,f"{label:.2f}",ha='center',va='center', c='black', fontsize = fs)
    
ax[1].axis('off')
ax[1].set_title(r'$P^n$' +f", n = {max_N}" )

# sumP_to_the_N
ax[2].imshow(sumP_to_the_N, cmap='Blues')

for (j,i),label in np.ndenumerate(sumP_to_the_N):
    ax[2].text(i,j,f"{label:.2f}",ha='center',va='center', c='black', fontsize = fs)
    
ax[2].axis('off')
ax[2].set_title(r'$\sum_{n=1}^{max_N} (P^n)$' +f", maxN = {max_N}" )


# pc.colorbar()
plt.tight_layout()

The expected number **explodes**, as you increase `max_N`!  
This is because you keep revisiting states over and over, indefinitely.  
- What happens if you use `P_red` (the reducible transition matrix)?
- What is a quantity that would make more sense tracking? (instead of the $N_i(j)$)

# 4. Find steady state distribution

**Method 1**: solving an eigenvector equation.

**Note**: you can sometime do this also _analytically_, by using a system of equations. This can be helpful in simple cases, e.g. when there are 2-3 states.

In [ ]:
ref_P = P.copy()
ref_P

In [ ]:
# Convert the transition matrix to a NumPy array
transition_matrix = np.array(ref_P)

# Calculate the steady-state distribution
eigenvalues, eigenvectors = np.linalg.eig(ref_P.T)
steady_state = eigenvectors[:, np.isclose(eigenvalues, 1)]

# Normalize the steady-state distribution
steady_state = steady_state / steady_state.sum()

print(steady_state.real.flatten())

**Method 2**: using the power of $P$


$ \lim_{n \rightarrow \infty} [P^n]_{ij}= \pi_{j}$

In [ ]:
max_N = 100
P_to_the_N = matrix_power(ref_P, max_N)

P_to_the_N[0]

In [ ]:
i = 0
assert np.allclose(P_to_the_N[i],steady_state.real.flatten(),rtol=1e-3)

**Method 3**: using the power of $P$ (and its relations with the expected number of visited sites)


$ \lim_{n \rightarrow \infty} \frac{\mathbb{E} [N_{n,i}(j)]}{n}= \pi_{j}$

In [ ]:
max_N = 10000
sumP_to_the_N = sum([matrix_power(ref_P, n)/max_N for n in range(1,max_N)])

sumP_to_the_N

In [ ]:
sumP_to_the_N[i]

In [ ]:
i = 0
assert np.allclose(sumP_to_the_N[i],steady_state.real.flatten(),rtol=1e-3)

## 5. Example 9.4
$P = \left[\begin{array}  
                     - 1- p & p\\
                    q & 1-q
                    \end{array}
                    \right]$  
                    
Let's consider this illustrative example for 2-state chains.  
By plaing with the values of $p,q$ we can see different behaviors.

In [ ]:
fs = 20
color = 'salmon'

#### Play with varying $p$ and $q$:
- a) $0 < p+q < 2$  
      - Example: $p=1.0$ and $q=0.5$
- b) $p=q=0$
- c) $p=q=1$
  

In [ ]:
p = 1.0
q = 0.5

In [ ]:
P_exa = np.array([[1-p,p],[q,1-q]])
assert np.all(np.isclose(np.sum(P_exa,1),1)) # check that is a valid row-sum to 1 matrix

plt.figure(figsize=(6,3))

# Using a matrix visualization

plt.subplot(1,2,1)
plt.imshow(P_exa, vmax=1,vmin=0, cmap='Blues')

for (j,i),label in np.ndenumerate(P_exa):
    plt.text(i,j,f"{label:.1f}",ha='center',va='center', c=color, fontsize = fs)
    plt.text(i,j,f"{label:.1f}",ha='center',va='center', c=color, fontsize = fs)
plt.axis('off')
plt.colorbar()

# Using a graph visualization
plt.subplot(1,2,2)
G = get_graph_from_transition(P_exa,['A','B'])

pos = nx.spring_layout(G, seed=10)

nx.draw_networkx_nodes(G, pos, node_size=1000, edgecolors='black', node_color='white')
nx.draw_networkx_labels(G, pos, font_size=12)

arc_rad = 0.2

ax = plt.gca()
edges = nx.draw_networkx_edges(G, pos, ax=ax, connectionstyle=f'arc3, rad = {arc_rad}', edge_cmap=cm.Blues, width=5,
    edge_color=[G[nodes[0]][nodes[1]][0]['weight'] for nodes in G.edges])

pc = mpl.collections.PatchCollection(edges, cmap=cm.Blues)
ax.set_axis_off()
plt.colorbar(pc, ax=ax)
plt.tight_layout()

In [ ]:
max_N = 1000

Limiting (equlibrium) distribution

$A=\lim_{n\rightarrow \infty} P^n$

In [ ]:

matrix_power(P_exa, max_N)

Steady state, using the expected sum:

$ \lim_{n \rightarrow \infty} \frac{\mathbb{E} [N_{n,i}(j)]}{n}= \pi_{j}$

In [ ]:

sum([matrix_power(P_exa, n)/max_N for n in range(1,max_N)])

In [ ]:
# Calculate the steady-state distribution
eigenvalues, eigenvectors = np.linalg.eig(P_exa.T)
steady_state = eigenvectors[:, np.isclose(eigenvalues, 1)]

# Normalize the steady-state distribution
steady_state = steady_state / steady_state.sum()

print(steady_state.real.flatten())